In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import numpy as np
import json

np.set_printoptions(precision=3, suppress=True)
# uncomment and replace with correct path if there are import errors
# sys.path.append("/path/to/surface-sampling/")
# sys.path.append("/path/to/NeuralForceField/")
# os.environ["LAMMPS_POTENTIALS"] = "/path/to/lammps/potentials/"
# os.environ["LAMMPS_COMMAND"] ="/path/to/lammps/src/lmp_serial"
# os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

import ase

from mcmc.system import SurfaceSystem
from mcmc.utils.misc import get_atoms_batch
import torch
from nff.io.ase_calcs import NeuralFF, AtomsBatch
from mcmc.calculators import EnsembleNFFSurface

/home/dux/NeuralForceField/models


Initialize test slab and parameters

In [3]:
# Load prepared pristine slab
element = []
slab_pkl = open("../tutorials/data/SrTiO3_001_2x2_pristine_slab.pkl", "rb")
slab = pickle.load(slab_pkl)
offset_data_path = os.path.join(
    "../tutorials",
    "data/nff",
    "offset_data.json",
)

ref_slab_files = [
    "data/SrTiO3_001/O44Sr12Ti16.cif",
    "data/SrTiO3_001/O36Sr12Ti12.cif",
    "data/SrTiO3_001/O40Sr16Ti12.cif",
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [4]:
system_settings = {
    "surface_name": "SrTiO3(001)",
    "cutoff": 5.0,
    "near_reduce": 0.01,
    "planar_distance": 1.55,
    "no_obtuse_hollow": True,
}

sampling_settings = {
    "alpha": 1.0,  # no annealing
    "temperature": 1.0,  # in terms of kbT
    "num_sweeps": 10,
    "sweep_size": 64,
}

calc_settings = {
    "calc_name": "NFF",
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0},
    "offset_data": json.load(open(offset_data_path, "r")),
    "optimizer": "BFGS",
    "relax_atoms": True,
    "relax_steps": 20,
    "offset": True,
}

Obtain adsorption sites

In [5]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = slab.copy()
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# new standardized params
ads_positions = site_finder.find_adsorption_sites(
    put_inside=True,
    symm_reduce=False,
    near_reduce=system_settings["near_reduce"],
    distance=system_settings["planar_distance"],
    no_obtuse_hollow=system_settings["no_obtuse_hollow"],
)["all"]

print("adsorption coordinates are")
print(ads_positions)

sweep_size = len(ads_positions)

adsorption coordinates are
[array([ 7.871,  7.941, 18.87 ]), array([ 1.968,  1.951, 18.829]), array([ 1.968,  0.146, 18.782]), array([ 7.871,  3.956, 18.87 ]), array([ 1.968,  5.936, 18.829]), array([ 1.968,  4.131, 18.782]), array([ 3.936,  7.941, 18.87 ]), array([ 5.903,  1.951, 18.829]), array([ 5.903,  0.146, 18.782]), array([ 3.936,  3.956, 18.87 ]), array([ 5.903,  5.936, 18.829]), array([ 5.903,  4.131, 18.782]), array([ 1.968,  7.026, 18.806]), array([ 0.984,  6.938, 18.85 ]), array([ 5.903,  7.026, 18.806]), array([ 6.887,  6.938, 18.85 ]), array([ 6.887,  0.058, 18.826]), array([ 7.871,  5.936, 18.829]), array([ 6.887,  4.946, 18.85 ]), array([ 0.984,  4.946, 18.85 ]), array([ 2.952,  6.938, 18.85 ]), array([ 6.887,  0.961, 18.85 ]), array([ 0.984,  0.961, 18.85 ]), array([ 7.871,  1.951, 18.829]), array([ 6.887,  4.043, 18.826]), array([ 5.903,  5.033, 18.806]), array([ 4.919,  4.946, 18.85 ]), array([ 4.919,  4.043, 18.826]), array([ 4.919,  0.058, 18.826]), array([ 4.919, 

Set up NFF (calculator). We are using neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file

In [6]:
if torch.cuda.is_available():
    DEVICE = 0
else:
    DEVICE = "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_settings)

offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters
chemical potentials: {'Sr': -2, 'Ti': 0, 'O': 0} are set from parameters
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


Initialize surface system

In [7]:
# set attributes
slab_batch = get_atoms_batch(
    slab,
    system_settings["cutoff"],
    DEVICE,
    props={"energy": 0, "energy_grad": []},
)

# fix bulk atoms
num_bulk_atoms = len(slab_batch)
bulk_indices = list(range(num_bulk_atoms))
print(f"bulk indices {bulk_indices}")
surf_indices = slab.get_surface_atoms()

fix_indices = list(set(bulk_indices) - set(surf_indices))
print(f"fix indices {fix_indices}")

# bulk atoms are 0 and surface atoms are 1
tags = np.ones(num_bulk_atoms)
tags[fix_indices] = 0
slab_batch.set_tags(tags)
print(f"tags {tags}")

surface = SurfaceSystem(
    slab_batch, ads_positions, nff_surf_calc, system_settings=system_settings
)
surface.all_atoms.write("SrTiO3_001_2x2_all_virtual_ads.cif")

2024-05-23 16:58:26,470|INFO|initializing 64 virtual atoms
2024-05-23 16:58:26,504|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-05-23 16:58:26,505|INFO|number of pristine atoms is 60
2024-05-23 16:58:26,505|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-05-23 16:58:26,506|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


bulk indices [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
fix indices [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58]
tags [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
      Step     Time          Energy         fmax
BFGS:    0 16:58:27     -467.521881        0.2306
BFGS:    1 16:58:27     -467.527832        0.1610
BFGS:    2 16:58:27     -467.533936        0.1001
BFGS:    3 16:58:27     -467.535400        0.1234
BFGS:    4 16:58:27     -467.539459        0.1376
BFGS:    5 16:58:27     -467.541412        0.0825
BFGS:    6 16:58:27  

In [8]:
surface.get_surface_energy(recalculate=True)

      Step     Time          Energy         fmax
BFGS:    0 16:58:28     -467.521881        0.2306
BFGS:    1 16:58:28     -467.527832        0.1610
BFGS:    2 16:58:28     -467.533936        0.1001
BFGS:    3 16:58:28     -467.535400        0.1234
BFGS:    4 16:58:28     -467.539459        0.1376
BFGS:    5 16:58:28     -467.541412        0.0825
BFGS:    6 16:58:28     -467.542236        0.0278
BFGS:    7 16:58:28     -467.542328        0.0162
BFGS:    8 16:58:29     -467.542389        0.0149
BFGS:    9 16:58:29     -467.542389        0.0126
BFGS:   10 16:58:29     -467.542450        0.0158
BFGS:   11 16:58:29     -467.542480        0.0177
BFGS:   12 16:58:29     -467.542511        0.0134
BFGS:   13 16:58:29     -467.542511        0.0088


array([12.469], dtype=float32)

Test out other reference slabs

In [9]:
ref_slab_batches = [
    AtomsBatch(
        positions=slab.positions,
        numbers=slab.numbers,
        cell=slab.cell,
        pbc=True,
        cutoff=system_settings["cutoff"],
        props={"energy": 0, "energy_grad": []},
        calculator=nff_surf_calc,
        requires_large_offsets=True,
        directed=True,
        device=DEVICE,
    )
    for slab in ref_slabs
]

ref_surfs = []
# additional atoms are adsorbates and should be labelled as 2
for ref_slab_batch in ref_slab_batches:
    num_ads = len(ref_slab_batch) - len(tags)
    curr_tags = np.int32(tags.tolist() + [2] * num_ads)
    print(f"tags {curr_tags}")
    ref_slab_batch.set_tags(curr_tags)
    ref_surf = SurfaceSystem(
        ref_slab_batch, ads_positions, nff_surf_calc, system_settings=system_settings
    )
    ref_surfs.append(ref_surf)

2024-05-23 16:58:31,324|INFO|initializing 64 virtual atoms
2024-05-23 16:58:31,366|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-05-23 16:58:31,367|INFO|number of pristine atoms is 72
2024-05-23 16:58:31,368|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-05-23 16:58:31,369|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2]
      Step     Time          Energy         fmax
BFGS:    0 16:58:31     -570.127991        1.3431
BFGS:    1 16:58:31     -570.247986        0.9791
BFGS:    2 16:58:31     -570.414429        0.7114
BFGS:    3 16:58:31     -570.452393        0.5440
BFGS:    4 16:58:31     -570.538391        0.3434
BFGS:    5 16:58:31     -570.551575        0.2750
BFGS:    6 16:58:32     -570.581787        0.2215
BFGS:    7 16:58:32     -570.586609        0.2149
BFGS:    8 16:58:32     -570.598877        0.1552
BFGS:    9 16:58:32     -570.603455        0.1359
BFGS:   10 16:58:32     -570.607605        0.1231
BFGS:   11 16:58:32     -570.611084        0.0945
BFGS:   12 16:58:32     -570.613892        0.0803
BFGS:   13 16:58:32     -570.615601        0.0690
BFGS:   14 16:58:32     -570.616882        0.0670
BFGS:   15 16:58:32     -570.617981        0.0537

2024-05-23 16:58:33,264|INFO|initializing 64 virtual atoms


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1]


2024-05-23 16:58:33,295|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-05-23 16:58:33,296|INFO|number of pristine atoms is 60
2024-05-23 16:58:33,296|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-05-23 16:58:33,297|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


      Step     Time          Energy         fmax
BFGS:    0 16:58:33     -467.525604        0.1979
BFGS:    1 16:58:33     -467.529663        0.1535
BFGS:    2 16:58:33     -467.535431        0.0958
BFGS:    3 16:58:33     -467.536652        0.1042
BFGS:    4 16:58:33     -467.540802        0.0906
BFGS:    5 16:58:33     -467.542084        0.0440
BFGS:    6 16:58:33     -467.542328        0.0186
BFGS:    7 16:58:33     -467.542358        0.0178
BFGS:    8 16:58:34     -467.542450        0.0143
BFGS:    9 16:58:34     -467.542450        0.0144
BFGS:   10 16:58:34     -467.542480        0.0179
BFGS:   11 16:58:34     -467.542511        0.0147
BFGS:   12 16:58:34     -467.542572        0.0101
BFGS:   13 16:58:34     -467.542572        0.0108
BFGS:   14 16:58:34     -467.542603        0.0088
tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2]


2024-05-23 16:58:34,602|INFO|initializing 64 virtual atoms
2024-05-23 16:58:34,634|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-05-23 16:58:34,635|INFO|number of pristine atoms is 68
2024-05-23 16:58:34,635|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-05-23 16:58:34,636|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


      Step     Time          Energy         fmax
BFGS:    0 16:58:34     -518.694092        0.9103
BFGS:    1 16:58:34     -518.770020        0.7165
BFGS:    2 16:58:34     -516.961365       20.6810
BFGS:    3 16:58:34     -518.799500        0.6865
BFGS:    4 16:58:35     -518.822571        0.6634
BFGS:    5 16:58:35     -518.509216        8.7417
BFGS:    6 16:58:35     -518.886597        0.6386
BFGS:    7 16:58:35     -518.911194        0.5211
BFGS:    8 16:58:35     -518.923889        0.6170
BFGS:    9 16:58:35     -518.929871        0.1595
BFGS:   10 16:58:35     -518.932068        0.1401
BFGS:   11 16:58:35     -518.939575        0.1379
BFGS:   12 16:58:35     -518.940979        0.1276
BFGS:   13 16:58:35     -518.944519        0.1541
BFGS:   14 16:58:35     -518.946472        0.2074
BFGS:   15 16:58:36     -518.948486        0.1819
BFGS:   16 16:58:36     -518.949646        0.0950
BFGS:   17 16:58:36     -518.950439        0.0723
BFGS:   18 16:58:36     -518.951111        0.0791
B

In [10]:
for surf in ref_surfs:
    surf_energy = surf.get_surface_energy()
    print(f"energy of reference slab is {surf_energy}")

energy of reference slab is [35.499]
energy of reference slab is [12.469]
energy of reference slab is [-5.045]
